<a href="https://colab.research.google.com/github/rkrisanoff/geographical-information-retrieval/blob/main/GIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install natasha

In [2]:
# news = glob.glob(f"/content/drive/MyDrive/GIR/FONTANKA-NEWS/news/*.txt")
# import glob
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from pprint import pprint

In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [5]:
import gdown
# Реальный датасет будет намного больше
shared_link = 'https://drive.google.com/drive/folders/1YM3wLKR9qJg1SuTTk9UG2iISEObSNtw0?usp=share_link'
output = 'news'
news = gdown.download_folder(url=shared_link,output=output,remaining_ok=True,quiet=True) 

In [ ]:
documents = dict()

for new_path in news[:5]:
  text = ""
  with open(new_path) as new:
    category,title,text = new.readlines()[:3]
    if title[-1:] == '\n':
      title = title[:-1]
  doc = Doc(text)
  doc.segment(segmenter) 
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  doc.tag_ner(ner_tagger)
  for span in doc.spans:
    span.normalize(morph_vocab)
  for span in doc.spans:
    span.extract_fact(names_extractor)

  documents[title] = []
  for span in doc.spans:
    if span.type == "LOC":
      documents[title].append(span.normal)

In [13]:
for title,words in documents.items():
  print("")
  print(title)
  print('!'+'-'*(len(title)-2)+'!')
  print(words)
  print('^'+'-'*(len(title)-2)+'^')


Губернатор Петербурга и замминистра транспорта летали над КАДом на вертолете
!--------------------------------------------------------------------------!
['Санкт-Петербург', 'РФ', 'Санкт-Петербург', 'РФ', 'Санкт-Петербург', 'Нева', 'Сосновское', 'Мурино', 'Муринский']
^--------------------------------------------------------------------------^

Кольцевая дорога "обовьется" вокруг бюджета
!-----------------------------------------!
['Петербург', 'Петербург', 'Москва', 'Горская', 'Мурино']
^-----------------------------------------^

Питерские прыжки на батуте удостоились "серебра"
!----------------------------------------------!
['Краснодар', 'Россия', 'Муриной']
^----------------------------------------------^

Полковник МЧС воровал взрывчатку килограммами
!-------------------------------------------!
['Москва', 'Мурино', 'Ленинградская область']
^-------------------------------------------^

Сотрудники УБОП закрыли подпольный водочный цех
!--------------------------------------------